## 가비지 컬렉션
- GC는 메모리를 자동으로 관리
- 사람이 직접 하는 것 보다는 최적화가 덜 되어있음
- Gabage Collection이 쓸모없어진 객체들을 잘 해제할수 있도록 레퍼런스 카운트에 신경을 써주어야 함
- 특히, 순환참조의 경우는 프로그램이 종료될때까지 메모리에 남아 있게 되므로 특히 주의

#### 객체 지우기

In [2]:
class A:
    def __del__(self):
        print("deleted")

a = A()
del(a)

deleted


#### 참조중인 객체 del 실행
- 이전과는 다르게 delete라는 메시지를 출력하지 않음
- 즉, a라는 객체가 메모리에서 지워지지 않았다는 얘기
- b라는 변수에서 a를 참조중이기 때문
- 메모리 누수가 발생할 수 있음

In [6]:
class A:
    def __del__(self):
        print("deleted")

a = A()
b = a

del(a)

####  참조된 횟수 확인 ( 레퍼런스 카운트 )

In [5]:
import sys

class A:
    pass

a = A()
b = a

print(sys.getrefcount(a))


3


#### 오브젝트가 현재 참조중인 목록 확인

In [7]:
import gc

def test():
    class A:
        pass

    class B:
        def __init__(self, obj): 
            self.obj = obj

    a = A()
    b = B(a)

    gc.collect() # make sure all garbage cleared before collecting referrers.    
    print( gc.get_referents(b))

test()

[{'obj': <__main__.test.<locals>.A object at 0x7fa9d0c625e0>}, <class '__main__.test.<locals>.B'>]


####  오브젝트를 참조중인 목록 확인

In [8]:
import gc

def test():
    class A:
        pass

    class B:
        def __init__(self, obj): 
            self.obj = obj

    a = A()
    b = B(a)

    gc.collect() # make sure all garbage cleared before collecting referrers.    
    print( gc.get_referrers(a) )

test()

[{'obj': <__main__.test.<locals>.A object at 0x7faa10ee8580>}]


#### 현재 자신을 참조중인 오브젝트를 찾아내어 강제로 자신을 지우기

In [10]:
import sys, gc

def delete_me(obj):
    referrers = gc.get_referrers(obj)
    for referrer in referrers:
        if type(referrer) == dict:
            for key, value in referrer.items():
                if value is obj:
                    referrer[key] = None

def test():            
    class A:
        def __del__(self):
                print("deleted")

    class B:
        def __init__(self, obj): 
            self.obj = obj

    a = A()
    b = B(a)

    print("before : ", b.__dict__)
    delete_me(a)
    print("after : ", b.__dict__)
    print("ref count : ", sys.getrefcount(a))
    gc.collect()
    print("ref count : ", sys.getrefcount(a))
    del(a)

test()

before :  {'obj': <__main__.test.<locals>.A object at 0x7faa10be5670>}
after :  {'obj': None}
ref count :  2
ref count :  2
deleted
